# Logistic回归 (Logistic Regression)

---

## 📚 学习目标

通过本教程，你将学会：
- 理解Logistic回归的原理和Sigmoid函数
- 掌握二分类和多分类问题的处理
- 学会使用Scikit-learn实现Logistic回归
- 理解决策边界和概率预测
- 掌握正则化在Logistic回归中的应用

## 📋 前置知识

- 线性回归基础
- 概率论基础
- 梯度下降概念

## ⏱️ 预计时间

30-40分钟

---

## 📖 第1部分：理论背景

### 什么是Logistic回归？

虽然名字里有"回归"，但**Logistic回归**实际上是一种**分类算法**。它通过Sigmoid函数将线性回归的输出映射到[0, 1]区间，表示样本属于某一类的概率。

### Sigmoid函数

$$\sigma(t) = \frac{1}{1 + e^{-t}}$$

特点：
- 输出范围：(0, 1)
- σ(0) = 0.5
- t → +∞ 时，σ(t) → 1
- t → -∞ 时，σ(t) → 0

### 模型公式

$$P(y=1|X) = \sigma(\theta^TX) = \frac{1}{1 + e^{-\theta^TX}}$$

### 损失函数（对数损失）

$$J(\theta) = -\frac{1}{m}\sum_{i=1}^{m}[y^{(i)}\log(\hat{p}^{(i)}) + (1-y^{(i)})\log(1-\hat{p}^{(i)})]$$

### 多分类：Softmax回归

对于K个类别，Softmax函数：

$$P(y=k|X) = \frac{e^{\theta_k^TX}}{\sum_{j=1}^{K}e^{\theta_j^TX}}$$

> 💡 **关键洞察**: Logistic回归是广义线性模型的一种，它输出的是概率而非确定性的类别标签。

---

## 💻 第2部分：代码实现

### 步骤1: 导入库和配置

In [ ]:
# ============================================================
# 导入必要的库
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# ============================================================
# 配置
# ============================================================

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

print("✓ 配置完成")

### 步骤2: 可视化Sigmoid函数

In [ ]:
# ============================================================
# Sigmoid函数可视化
# ============================================================

def sigmoid(t):
    """Sigmoid函数"""
    return 1 / (1 + np.exp(-t))

t = np.linspace(-10, 10, 100)

plt.figure(figsize=(10, 4))

# Sigmoid曲线
plt.subplot(1, 2, 1)
plt.plot(t, sigmoid(t), 'b-', linewidth=2)
plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.5, label='y=0.5')
plt.axvline(x=0, color='g', linestyle='--', alpha=0.5, label='x=0')
plt.xlabel('t')
plt.ylabel('σ(t)')
plt.title('Sigmoid函数', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(-0.1, 1.1)

# 决策规则
plt.subplot(1, 2, 2)
plt.fill_between(t[t<0], 0, sigmoid(t[t<0]), alpha=0.3, color='blue', label='预测为0')
plt.fill_between(t[t>=0], 0, sigmoid(t[t>=0]), alpha=0.3, color='red', label='预测为1')
plt.plot(t, sigmoid(t), 'k-', linewidth=2)
plt.axvline(x=0, color='green', linestyle='--', linewidth=2, label='决策边界')
plt.xlabel('t = θᵀX')
plt.ylabel('P(y=1)')
plt.title('决策规则', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("决策规则:")
print("  - 当 θᵀX ≥ 0 时，预测为类别1 (P ≥ 0.5)")
print("  - 当 θᵀX < 0 时，预测为类别0 (P < 0.5)")

---

## 🌸 第3部分：二分类 - 鸢尾花数据

### 加载数据

In [ ]:
# ============================================================
# 加载鸢尾花数据集
# ============================================================

iris = datasets.load_iris()

print("数据集信息:")
print(f"  样本数: {iris.data.shape[0]}")
print(f"  特征数: {iris.data.shape[1]}")
print(f"  特征名: {iris.feature_names}")
print(f"  类别名: {iris.target_names}")

### 简单二分类：花瓣宽度预测是否为Virginica

In [ ]:
# ============================================================
# 二分类问题
# ============================================================

# 使用花瓣宽度（第4个特征）
X = iris.data[:, 3:]  # petal width

# 二分类：是否为Virginica (类别2)
y = (iris.target == 2).astype(int)

print(f"特征: {iris.feature_names[3]}")
print(f"类别分布: 非Virginica={sum(y==0)}, Virginica={sum(y==1)}")

In [ ]:
# ============================================================
# 训练Logistic回归
# ============================================================

# 创建并训练模型
log_reg = LogisticRegression(random_state=RANDOM_SEED)
log_reg.fit(X, y)

print("模型参数:")
print(f"  截距: {log_reg.intercept_[0]:.4f}")
print(f"  系数: {log_reg.coef_[0][0]:.4f}")

In [ ]:
# ============================================================
# 可视化概率曲线和决策边界
# ============================================================

# 生成预测用数据
X_new = np.linspace(0, 3, 1000).reshape(-1, 1)
y_proba = log_reg.predict_proba(X_new)

# 找到决策边界（P=0.5的点）
decision_boundary = X_new[np.argmin(np.abs(y_proba[:, 1] - 0.5))][0]

plt.figure(figsize=(12, 5))

# 左图：概率曲线
plt.subplot(1, 2, 1)
plt.plot(X_new, y_proba[:, 1], 'g-', linewidth=2, label='P(Virginica)')
plt.plot(X_new, y_proba[:, 0], 'b--', linewidth=2, label='P(非Virginica)')
plt.axvline(x=decision_boundary, color='r', linestyle='--', linewidth=2, 
            label=f'决策边界 x={decision_boundary:.2f}')
plt.axhline(y=0.5, color='gray', linestyle=':', alpha=0.5)
plt.xlabel('花瓣宽度 (cm)')
plt.ylabel('概率')
plt.title('概率预测曲线', fontweight='bold')
plt.legend(loc='center right')
plt.grid(True, alpha=0.3)

# 右图：散点图与决策边界
plt.subplot(1, 2, 2)
plt.scatter(X[y==0], y[y==0], c='blue', marker='o', s=50, label='非Virginica', alpha=0.6)
plt.scatter(X[y==1], y[y==1], c='green', marker='s', s=50, label='Virginica', alpha=0.6)
plt.axvline(x=decision_boundary, color='r', linestyle='--', linewidth=2, label='决策边界')
plt.xlabel('花瓣宽度 (cm)')
plt.ylabel('类别')
plt.title('数据分布与决策边界', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n决策边界: 花瓣宽度 = {decision_boundary:.2f} cm")

In [ ]:
# ============================================================
# 预测示例
# ============================================================

# 测试预测
test_samples = [[1.7], [1.5], [0.5], [2.5]]

print("预测示例:")
print(f"{'花瓣宽度':>10} {'预测类别':>10} {'P(Virginica)':>15}")
print("-" * 40)

for sample in test_samples:
    pred = log_reg.predict([sample])[0]
    prob = log_reg.predict_proba([sample])[0][1]
    label = 'Virginica' if pred == 1 else '非Virginica'
    print(f"{sample[0]:>10.1f} {label:>10} {prob:>15.4f}")

---

## 🌈 第4部分：多分类 - Softmax回归

### 使用两个特征进行三分类

In [ ]:
# ============================================================
# 多分类问题
# ============================================================

# 使用花瓣长度和花瓣宽度两个特征
X = iris.data[:, (2, 3)]  # petal length, petal width
y = iris.target  # 3个类别

# 划分数据
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=RANDOM_SEED, stratify=y
)

print(f"训练集: {len(X_train)} 样本")
print(f"测试集: {len(X_test)} 样本")
print(f"类别: {iris.target_names}")

In [ ]:
# ============================================================
# 训练Softmax回归
# ============================================================

# 创建多分类Logistic回归
# C是正则化强度的倒数，C越大正则化越弱
softmax_reg = LogisticRegression(
    C=10,                    # 正则化参数（1/C是正则化强度）
    max_iter=1000,
    random_state=RANDOM_SEED
)

softmax_reg.fit(X_train, y_train)

# 评估
y_pred = softmax_reg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"测试集准确率: {accuracy:.4f}")

In [ ]:
# ============================================================
# 可视化决策边界
# ============================================================

def plot_decision_boundary(model, X, y, ax=None, title=''):
    """绘制决策边界"""
    if ax is None:
        ax = plt.gca()
    
    # 创建网格
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))
    
    # 预测网格点
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # 绘制决策区域
    ax.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.RdYlBu)
    ax.contour(xx, yy, Z, colors='k', linewidths=0.5)
    
    # 绘制数据点
    scatter = ax.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu, 
                        edgecolors='k', s=50, alpha=0.8)
    
    ax.set_xlabel('花瓣长度 (cm)')
    ax.set_ylabel('花瓣宽度 (cm)')
    ax.set_title(title, fontweight='bold')
    
    return scatter

plt.figure(figsize=(12, 5))

# 训练集
plt.subplot(1, 2, 1)
plot_decision_boundary(softmax_reg, X_train, y_train, 
                       title=f'训练集 (准确率: {softmax_reg.score(X_train, y_train):.2%})')

# 测试集
plt.subplot(1, 2, 2)
scatter = plot_decision_boundary(softmax_reg, X_test, y_test,
                                title=f'测试集 (准确率: {accuracy:.2%})')

# 添加图例
plt.legend(handles=scatter.legend_elements()[0], labels=list(iris.target_names),
          loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 概率预测示例
# ============================================================

# 预测特定样本
test_sample = [[5, 2]]

pred_class = softmax_reg.predict(test_sample)[0]
pred_proba = softmax_reg.predict_proba(test_sample)[0]

print(f"\n样本 [花瓣长度=5cm, 花瓣宽度=2cm] 的预测:")
print(f"  预测类别: {iris.target_names[pred_class]}")
print(f"\n  各类别概率:")
for i, (name, prob) in enumerate(zip(iris.target_names, pred_proba)):
    bar = '█' * int(prob * 30)
    print(f"    {name:>12}: {prob:.4f} {bar}")

---

## 📊 第5部分：模型评估

In [ ]:
# ============================================================
# 详细评估
# ============================================================

# 混淆矩阵
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 4))

# 混淆矩阵热图
plt.subplot(1, 2, 1)
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('混淆矩阵', fontweight='bold')
plt.colorbar()

tick_marks = np.arange(len(iris.target_names))
plt.xticks(tick_marks, iris.target_names, rotation=45)
plt.yticks(tick_marks, iris.target_names)

# 在格子中显示数值
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], ha='center', va='center',
                color='white' if cm[i, j] > cm.max()/2 else 'black')

plt.xlabel('预测类别')
plt.ylabel('真实类别')

# 各类别预测概率分布
plt.subplot(1, 2, 2)
y_proba_test = softmax_reg.predict_proba(X_test)
for i, name in enumerate(iris.target_names):
    plt.hist(y_proba_test[y_test==i, i], bins=10, alpha=0.5, label=name)
plt.xlabel('预测概率')
plt.ylabel('样本数')
plt.title('各类别正确预测概率分布', fontweight='bold')
plt.legend()

plt.tight_layout()
plt.show()

# 分类报告
print("\n分类报告:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))

---

## 🔧 第6部分：正则化参数C的影响

In [ ]:
# ============================================================
# 不同C值对决策边界的影响
# ============================================================

C_values = [0.01, 0.1, 1, 10, 100]

fig, axes = plt.subplots(1, 5, figsize=(18, 4))

for ax, C in zip(axes, C_values):
    model = LogisticRegression(C=C, max_iter=1000, random_state=RANDOM_SEED)
    model.fit(X_train, y_train)
    
    acc = model.score(X_test, y_test)
    plot_decision_boundary(model, X_train, y_train, ax=ax, 
                          title=f'C={C}\n准确率: {acc:.2%}')

plt.tight_layout()
plt.show()

print("\n观察结论:")
print("  - C小：强正则化，决策边界更平滑")
print("  - C大：弱正则化，决策边界更复杂")

---

## 📝 总结

### 关键要点

1. ✅ **Logistic回归**是分类算法，不是回归算法
2. ✅ 通过**Sigmoid函数**输出概率
3. ✅ **Softmax**是Logistic回归在多分类上的推广
4. ✅ 参数**C**控制正则化强度（C越大，正则化越弱）
5. ✅ 输出概率使得模型具有良好的可解释性

### Logistic回归的优缺点

| 优点 | 缺点 |
|------|------|
| 简单高效 | 只能处理线性可分问题 |
| 输出概率 | 特征工程依赖强 |
| 可解释性强 | 难以处理复杂关系 |
| 不易过拟合 | 对异常值敏感 |

### 应用场景

- 信用评分
- 疾病诊断
- 垃圾邮件检测
- 广告点击预测

### 下一步学习

- 📘 支持向量机 (SVM)
- 📙 决策树和随机森林
- 📕 神经网络

### 练习题

1. 使用全部4个特征训练模型，比较性能
2. 尝试不同的正则化类型（L1、L2）
3. 在其他数据集上应用Logistic回归

## 📚 参考资料

- [Scikit-learn: LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
- 《机器学习实战》第4章 - 训练模型

---

## 🧪 单元测试 | Unit Tests

In [ ]:
# ============================================================
# 单元测试：验证Logistic回归实现的正确性
# ============================================================

def run_tests():
    """运行所有单元测试"""
    print("🧪 开始单元测试...\n")
    passed = 0
    failed = 0
    
    # 测试1: Sigmoid函数验证
    try:
        assert abs(sigmoid(0) - 0.5) < 1e-10, "sigmoid(0) 应该等于 0.5"
        assert sigmoid(100) > 0.99, "sigmoid(大正数) 应该接近 1"
        assert sigmoid(-100) < 0.01, "sigmoid(大负数) 应该接近 0"
        print("✓ 测试1通过: Sigmoid函数正确")
        passed += 1
    except AssertionError as e:
        print(f"✗ 测试1失败: {e}")
        failed += 1
    
    # 测试2: 二分类模型参数验证
    try:
        assert log_reg.coef_.shape == (1, 1), f"系数形状错误: {log_reg.coef_.shape}"
        assert len(log_reg.intercept_) == 1, "截距数量错误"
        print("✓ 测试2通过: 二分类模型参数形状正确")
        passed += 1
    except AssertionError as e:
        print(f"✗ 测试2失败: {e}")
        failed += 1
    
    # 测试3: 概率和等于1
    try:
        test_proba = log_reg.predict_proba([[1.5]])
        assert abs(test_proba.sum() - 1.0) < 1e-10, "概率和不等于1"
        print("✓ 测试3通过: 概率和验证（二分类）")
        passed += 1
    except AssertionError as e:
        print(f"✗ 测试3失败: {e}")
        failed += 1
    
    # 测试4: 多分类模型准确率验证
    try:
        assert accuracy >= 0.8, f"准确率过低: {accuracy}"
        print(f"✓ 测试4通过: 多分类准确率 = {accuracy:.4f} >= 0.8")
        passed += 1
    except AssertionError as e:
        print(f"✗ 测试4失败: {e}")
        failed += 1
    
    # 测试5: Softmax概率和验证
    try:
        test_proba_multi = softmax_reg.predict_proba([[5, 2]])
        assert abs(test_proba_multi.sum() - 1.0) < 1e-10, "Softmax概率和不等于1"
        print("✓ 测试5通过: Softmax概率和验证")
        passed += 1
    except AssertionError as e:
        print(f"✗ 测试5失败: {e}")
        failed += 1
    
    # 测试6: 混淆矩阵形状验证
    try:
        assert cm.shape == (3, 3), f"混淆矩阵形状错误: {cm.shape}"
        assert cm.sum() == len(y_test), "混淆矩阵元素和不等于测试样本数"
        print("✓ 测试6通过: 混淆矩阵验证")
        passed += 1
    except AssertionError as e:
        print(f"✗ 测试6失败: {e}")
        failed += 1
    
    # 测试7: 决策边界在合理范围
    try:
        assert 1.0 < decision_boundary < 2.0, f"决策边界异常: {decision_boundary}"
        print(f"✓ 测试7通过: 决策边界 = {decision_boundary:.2f} 在合理范围")
        passed += 1
    except AssertionError as e:
        print(f"✗ 测试7失败: {e}")
        failed += 1
    
    # 总结
    print(f"\n{'='*50}")
    print(f"测试完成: {passed} 通过, {failed} 失败")
    if failed == 0:
        print("🎉 所有测试通过！代码实现正确。")
    else:
        print("⚠️ 存在失败的测试，请检查代码。")
    print(f"{'='*50}")
    
    return failed == 0

# 运行测试
run_tests()